<a href="https://colab.research.google.com/github/Akshay-ramagiri/NNDL/blob/main/Assignment5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##MOUNT GOOGLE DRIVE

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##IMPORTING REQUIRED PACKAGES

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

##IMPORT DATA

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Assignment5.csv')
data.head()

,Date,Time,Load (kW)
0,01.09.2018,0:00,5551.82208
1,NaN,1:00,4983.17184
2,NaN,2:00,4888.39680
3,NaN,3:00,5072.95872
4,NaN,4:00,5196.25980


In [ ]:
X = []
Y = []
for i in range(24, data.shape[0]):
    X.append(data['Load (kW)'].iloc[i-25]) 
    Y.append(data['Load (kW)'].iloc[i])

In [ ]:
data = pd.DataFrame({'Previous Day Load' : X, 'Present Day Load' : Y})
data.head()

,Previous Day Load,Present Day Load
0,6938.37468,4931.26380
1,5551.82208,4775.53968
2,4983.17184,4713.81120
3,4888.39680,4689.02628
4,5072.95872,4844.75040


##NORMALIZATION

In [ ]:
Xmax = np.max(data['Previous Day Load'])
Xmin = np.min(data['Previous Day Load'])
Ymax = np.max(data['Present Day Load'])
Ymin = np.min(data['Present Day Load'])
data['Previous Day Load'] = (data['Previous Day Load'] - Xmin) / (Xmax - Xmin)
data['Present Day Load'] = (data['Present Day Load'] - Ymin) / (Ymax - Ymin)
data.head()

,Previous Day Load,Present Day Load
0,0.651650,0.284300
1,0.397877,0.255799
2,0.293800,0.244501
3,0.276454,0.239965
4,0.310234,0.268466


##DATA SPLIT

In [ ]:
x = data['Previous Day Load']
y = data['Present Day Load']
Xtrain, Xtest, Ytrain, Ytest = train_test_split(x, y, test_size=0.1, random_state=101)

##GRADIENT CALCULATION AND MODEL TRAINING

In [ ]:
m = np.random.uniform(-2, 2)
c = np.random.uniform(-2, 2)
eta = 0.1
epochs = 500
gm = 0.9
Em2 = 0
Ec2 = 0
e = 1E-6
for _ in range(epochs):
    for i in range(Xtrain.shape[0]):
        gradM = -1 * (Ytrain.iloc[i] - m * Xtrain.iloc[i] - c) * Xtrain.iloc[i]
        gradC = -1 * (Ytrain.iloc[i] - m * Xtrain.iloc[i] - c)
        
        Em2 = (gm*Em2) + ((1-gm)*(gradM ** 2))
        Ec2 = (gm*Ec2) + ((1-gm)*(gradC ** 2))
        
        m -= (eta * gradM) / ((e + Em2) ** 0.5)
        c -= (eta * gradC) / ((e + Ec2) ** 0.5)

print(f'm = {m}\nc = {c}')

m = 0.7459993025267385
c = 0.14754753492225675


In [ ]:
train_pred = [m * Xtrain.iloc[i] + c for i in range(Xtrain.shape[0])]
test_pred = [m * Xtest.iloc[i] + c for i in range(Xtest.shape[0])]

##DENORMALIZATION

In [ ]:
train_pred = [i * (Ymax - Ymin) + Ymin for i in train_pred]
test_pred = [i * (Ymax - Ymin) + Ymin for i in test_pred]
trainY = [i * (Ymax - Ymin) + Ymin for i in Ytrain]
testY = [i * (Ymax - Ymin) + Ymin for i in Ytest]

##ERROR METRIC CALCULATION
TRAINING DATA

In [ ]:
train_MAE = sum([abs(train_pred[i] - trainY[i]) for i in range(len(trainY))]) / len(trainY)
train_MSE = sum([(train_pred[i] - trainY[i]) ** 2 for i in range(len(trainY))]) / len(trainY)
train_RMSE = train_MSE ** 0.5
print('Training Error :')
print(f'MAE : {train_MAE}')
print(f'MSE : {train_MSE}')
print(f'RMSE : {train_RMSE}')

Training Error :
MAE : 631.4166200569462
MSE : 691375.3194148488
RMSE : 831.4898191889356


TESTING DATA

In [ ]:
test_MAE = sum([abs(test_pred[i] - testY[i]) for i in range(len(testY))]) / len(testY)
test_MSE = sum([(test_pred[i] - testY[i]) ** 2 for i in range(len(testY))]) / len(testY)
test_RMSE = test_MSE ** 0.5
print('Testing Error :')
print(f'MAE : {test_MAE}')
print(f'MSE : {test_MSE}')
print(f'RMSE : {test_RMSE}')

Testing Error :
MAE : 613.0434535946625
MSE : 662035.6945268413
RMSE : 813.6557592291972


##Analyze training data predictions

In [ ]:
pd.DataFrame({'Actual' : trainY, 'Predicted':train_pred})

,Actual,Predicted
0,4221.85392,4724.814053
1,7609.43808,6742.382594
2,5847.05880,6601.676085
3,4672.50300,5074.254598
4,7883.78688,6141.182055
...,...,...
1939,5830.37964,5800.230414
1940,6110.49600,6717.497311
1941,5289.94368,6117.343349
1942,7962.97392,7279.625630


##Analyze testing data predictions

In [ ]:
pd.DataFrame({'Actual' : testY, 'Predicted':test_pred})

,Actual,Predicted
0,6161.624640,6185.370876
1,5761.792440,6040.245485
2,7390.270800,6496.088060
3,4597.368840,6404.919545
4,5242.712040,5936.052896
...,...,...
211,6764.319072,7340.792261
212,7390.270800,6588.884584
213,6550.077600,6512.135579
214,5722.043040,5488.815512


##PREDICTION

In [ ]:
x = float(input('Enter the load at an hour before of previous day : '))
x = (x - Xmin) / (Xmax - Xmin)
pred = m * x + c
pred = (pred * (Ymax - Ymin)) + Ymin
print('The Predicted load at present hour = ', pred)

Enter the load at an hour before of previous day : 5000
The Predicted load at present hour =  5394.153273268069
